### This program is an attempt to solve a sudoku puzzle using a genetic algorthim

In [1]:
import numpy as np
import random

In [2]:
def get_parents(dist, pop):
    pop_size = len(pop)
    dist_size = len(dist)
    split = int(pop_size/dist_size)
    
    list_idx = np.arange(dist_size)
    
    group_indices = np.random.choice(a=list_idx, size=2, p=dist)
    p1_group, p2_group = group_indices[0], group_indices[1]
    
    split_pop = [pop[i * split:(i + 1) * split] for i in range((len(pop) + split - 1) // split)]
    group1 = split_pop[p1_group]
    group2 = split_pop[p2_group]
    
    
    if p1_group == p2_group:
        indices = np.random.choice(a=len(group1), size=2, replace=False)
        p1_idx, p2_idx = indices[0], indices[1]
    else:
        p1_idx = np.random.choice(len(group1),size=1)[0]
        p2_idx = np.random.choice(len(group2),size=1)[0]
        
    p1 = group1[p1_idx]
    p2 = group2[p2_idx]
        
    return p1, p2

In [3]:
'''
This function is used to build a predefined number of sudoku boards. It retains the original layout and
fills in the blank squares with a random number between 1 and the length of the board.

init_board => initial board setup with given values and 0s representing blank squares
zeros => list of x and y coordinates where the initial board had blanks
pop_size => number of randomly initialized boards
'''
def build_pop(init_board, zeros, pop_size=100):
    h,_ = init_board.shape
    pop = []
    
    for i in range(pop_size):
        temp_board = init_board.copy()   # copy of original layout
        
        # fill in blanks with random numbers within the range of the board size
        for x,y in zeros:
            val = np.random.randint(low=1,high=(h+1))
            temp_board[x,y] = val
    
        pop.append(temp_board)
    
    return pop

In [4]:
'''
This function is used to calculate the fitness of a board. The goal of sudoku is to have no repeating number 
in each row, column, and subgrid. Here we define the fitness as the number of repeating digits in a row, column or subgrid.
In this case the lower the fitness, the better the board.
ex. with a 4x4 board:
worst              best                worst  [1]   best   [1]    worst  [1 1]   best   [1 2]
case = [1 1 1 1]   case = [1 2 3 4]    case = [1]   case = [2]    case = [1 1]   case = [3 4]
row                row                 col    [1]   col    [3]    grid           grid
                                              [1]          [4]
wosrt case fitness = size*(size-1)*3

board => board whose fitness is being measured
'''
def fitness(board):
    h,_ = board.shape   # length of each row and col
    grid_size = int(np.sqrt(h))   # len & width of side subgrid
    f = 0   # initialize fitness score
    
    # check every row
    for i in range(h):
        row = np.array(board[i,:])[0]
        f += h - len(np.unique(row))
    
    # check every col
    for i in range(h):
        col = np.array(board[:,i])
        col_t = np.transpose(col)
        f += h - len(np.unique(col_t))
        
    # check every subgrid
    col_split = np.split(board, grid_size, axis=1)
    
    for c in col_split:
        grid = np.split(c, grid_size)
        
        for g in grid:
            g_list = np.array(g.flatten())[0]
            f += h - len(np.unique(g_list))
    
    return f

In [5]:
'''
Return a population sorted by fitness().

pop => population
'''
def evaluation(pop):
    return sorted(pop, key = lambda x: fitness(x))

In [6]:
'''
Return top half of population.

pop => population
'''
def selection(pop):
    return pop[:len(pop)//2]

In [7]:
'''
This function takes in the new population, randomly selects two parents from the population and crosses between the two
in order to create new child boards.

pop => population
elitism => flag to indicate if parents are kept
'''
def crossover(pop, p_dist, elitism=False):
    pop_size = len(pop)
    children = []
    
    for i in range(pop_size):
        p1,p2 = get_parents(p_dist, pop)
        h, w = p1.shape
        board_len = p1.size
        
#         print(f'Shape: {h},{w}')
#         print(f'Board length: {board_len}')
        
        flat_p1 = p1.flatten()
        flat_p2 = p2.flatten()
#         print(f'Parent 1: {p1}')
#         print(f'Parent 2: {p2}')
#         print(f'Flat Parent 1: {flat_p1}')
#         print(f'Flat Parent 2: {flat_p2}')
        
#         pivot = np.random.randint(low=0, high=board_len)
        sample_vals = np.arange(board_len).tolist()
        pivot = random.sample(sample_vals, k=2)
        pivot1 = min(pivot)
        pivot2 = max(pivot)
        
        
        p1_first = flat_p1[0,:pivot1]
        p2_first = flat_p2[0,:pivot1]
        p1_middle = flat_p1[0,pivot1:pivot2]
        p2_middle = flat_p2[0,pivot1:pivot2]
        p1_last = flat_p1[0,pivot2:]
        p2_last = flat_p2[0,pivot2:]
        
#         print(f'Printing pivot: {pivot}')
        
#         p1_first = flat_p1[0,:pivot]
#         p1_last = flat_p1[0,pivot:]
#         p2_first = flat_p2[0,:pivot]
#         p2_last = flat_p2[0,pivot:]
        
#         print(f'First half of p1: {p1_first}')
#         print(f'Last half of p1: {p1_last}')
#         print(f'First half of p2: {p2_first}')
#         print(f'Last half of p2: {p2_last}')
        
        p1_first = np.array(p1_first)[0]
        p1_middle = np.array(p1_middle)[0]
        p1_last = np.array(p1_last)[0]
        p2_first = np.array(p2_first)[0]
        p2_middle = np.array(p2_middle)[0]
        p2_last = np.array(p2_last)[0]
        
#         print('Now np arrays')
#         print(f'First half of p1: {p1_first}')
#         print(f'Last half of p1: {p1_last}')
#         print(f'First half of p2: {p2_first}')
#         print(f'Last half of p2: {p2_last}')
        
        child1_flat = np.concatenate((p1_first,p2_middle,p1_last))
        child2_flat = np.concatenate((p2_first,p1_middle,p2_last))
        child1 = child1_flat.reshape(h,w)
        child2 = child2_flat.reshape(h,w)
        child1 = np.matrix(child1)
        child2 = np.matrix(child2)
        
#         print(f'Child 1: {child1}')
#         print(f'Child 2: {child2}')
        
        children.append(child1)
        if not elitism:
            children.append(child2)
#         print()
            
    if elitism:
        return pop+children
    else:
        return children

In [9]:
def mutation(pop,m,zeros):
    mutated_population = []
    for candidate in pop:
        if random.random() < m:
            size = len(candidate) + 1
            mutated_val = np.random.randint(low=1,high=size)
            mutated_candidate = candidate.copy()
#             k = np.random.choice(a=[1,2,3], size=1, p=[.75,.20,.05])[0]   # Not great
#             k = np.random.choice(a=[1,2,3], size=1, p=[.65,.25,.10])[0]   # used on easy board
            k = np.random.choice(a=[1,2,3], size=1, p=[.33,.34,.33])[0]   # testing on hard board
            indices = random.sample(zeros,k=k)
            for x,y in indices:
                mutated_candidate[x,y] = mutated_val
            
#             x, y = random.sample(zeros,k=1)[0]
#             mutated_candidate[x,y] = mutated_val
        else:
            mutated_candidate = candidate.copy()
        mutated_population.append(mutated_candidate)
    return mutated_population

#### First we will start out by defining the initial board setup. 
#### Zeros will be used to represent blank spaces on the board.

In [12]:
%%time
# init_board = np.matrix([[3,0,4,0],
#                         [0,1,0,2],
#                         [0,4,0,3],
#                         [2,0,1,0]])

# solved_test = np.matrix([[3,2,4,1],
#                          [4,1,3,2],
#                          [1,4,2,3],
#                          [2,3,1,4]])

# init_board = np.matrix([[5,3,0,0,7,0,0,0,0],
#                         [6,0,0,1,9,5,0,0,0],
#                         [0,9,8,0,0,0,0,6,0],
#                         [8,0,0,0,6,0,0,0,3],
#                         [4,0,0,8,0,3,0,0,1],
#                         [7,0,0,0,2,0,0,0,6],
#                         [0,6,0,0,0,0,2,8,0],
#                         [0,0,0,4,1,9,0,0,5],
#                         [0,0,0,0,8,0,0,7,9]])

# easy 3:44
# init_board = np.matrix([[4,5,0,8,7,2,9,0,0],
#                         [0,8,6,9,5,0,0,7,0],
#                         [0,0,2,0,0,0,4,8,5],
#                         [2,0,0,0,0,9,1,0,4],
#                         [6,0,9,0,4,0,8,0,2],
#                         [0,0,5,1,2,0,0,0,0],
#                         [0,6,4,0,0,8,0,0,9],
#                         [0,7,0,0,0,0,6,2,3],
#                         [9,0,0,7,0,1,0,0,0]])

init_board = np.matrix([[4,0,0,0,0,0,0,0,0],
                        [0,7,5,0,0,0,0,0,0],
                        [0,0,0,0,5,0,8,6,0],
                        [0,0,0,0,0,2,1,0,0],
                        [0,0,0,4,0,0,6,0,0],
                        [2,0,0,0,7,1,0,4,3],
                        [0,5,0,0,8,7,0,0,0],
                        [0,9,1,0,6,0,0,7,0],
                        [0,0,2,0,0,0,0,5,0]])



# find all zeros (blanks) on the board
x,y = np.where(init_board==0)
zeros = list(zip(x,y))

board_len = init_board.shape[0]
best_fitness = board_len*(board_len-1)*3
# p_dist = [0.4,0.2,0.2,0.1,0.1]
# p_dist = [0.5,0.2,0.15,0.1,0.05]
# p_dist = [0.65,0.15,0.1,0.05,0.05]   # best 
p_dist = [1.0,0.0,0.0,0.0,0.0]   # bad practice

pop = build_pop(init_board, zeros, 150)
pop = evaluation(pop)


i = 0
while best_fitness > 0:
    
    mutation_rate = 1.0
#     if best_fitness >= 15:
#         mutation_rate = .3
#     if best_fitness > 10:
#         mutation_rate = .5
#     else:
#         mutation_rate = .6
    
    pop = selection(pop)
#     pop = crossover(pop,p_dist,elitism=elitism)
    pop = crossover(pop,p_dist,elitism=True)
    pop = mutation(pop,mutation_rate,zeros)
#     pop = mutation(pop,.3,zeros)
    pop = evaluation(pop)
    
    if i % 500 == 0:
        f = [fitness(b) for b in pop]
        f = np.array(f)
        print(f'Averge board fitness at {i} = {np.average(f)}')
    
    if fitness(pop[0]) < best_fitness:
        best_fitness = fitness(pop[0])
        print(f"New best fitness: {best_fitness} | i={i}")
    i+=1
    
#     break

print(pop[0])

Averge board fitness at 0 = 74.378
New best fitness: 60 | i=0
New best fitness: 57 | i=1
New best fitness: 56 | i=2
New best fitness: 51 | i=3
New best fitness: 50 | i=4
New best fitness: 46 | i=5
New best fitness: 45 | i=6
New best fitness: 44 | i=7
New best fitness: 42 | i=9
New best fitness: 38 | i=10
New best fitness: 36 | i=13
New best fitness: 35 | i=14
New best fitness: 34 | i=15
New best fitness: 32 | i=17
New best fitness: 31 | i=19
New best fitness: 29 | i=21
New best fitness: 28 | i=23
New best fitness: 26 | i=24
New best fitness: 25 | i=28
New best fitness: 24 | i=29
New best fitness: 23 | i=31
New best fitness: 21 | i=32
New best fitness: 19 | i=37
New best fitness: 18 | i=41
New best fitness: 17 | i=44
New best fitness: 16 | i=47
New best fitness: 15 | i=50
New best fitness: 14 | i=56
New best fitness: 13 | i=57
New best fitness: 12 | i=63
New best fitness: 11 | i=68
New best fitness: 10 | i=83


KeyboardInterrupt: 